In [28]:
# Boltzmann Machines using pytorch

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [29]:
# Preparing the training set and the test set
training_set = pd.read_csv('ml-1m/training_set.csv')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-1m/test_set.csv')
test_set = np.array(test_set, dtype = 'int')

In [30]:
print("Training set is:\n",training_set)
print('\n\n\n')
print("Test set is:\n",test_set)

Training set is:
 [[        1       914         3 978301968]
 [        1      3408         4 978300275]
 [        1      2355         5 978824291]
 ...
 [     6040       562         5 956704746]
 [     6040      1096         4 956715648]
 [     6040      1097         4 956715569]]




Test set is:
 [[        1      1193         5 978300760]
 [        1      1197         3 978302268]
 [        1      2804         5 978300719]
 ...
 [     6040       527         5 956704219]
 [     6040      2003         1 956716294]
 [     6040       535         4 964828734]]


In [31]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(nb_users)
print(nb_movies)

6040
3952


In [32]:
# Converting the data into an array with users in rows and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [33]:
# Call convert 
training_set = convert(training_set)
test_set = convert(test_set)

In [34]:
print(training_set[0][:10])
print(test_set[0][:10])

[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [35]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [36]:
print('Training set:\n', training_set)
print('Test set:\n', test_set)

Training set:
 tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.]])
Test set:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [37]:
# Converting the ratings 1-5 into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [38]:
# Creating the architecture of the Neural Network
class RBM():
    # Initialize Weights W of shape no. of visible nodes times no. of hidden nodes
    # a is the bais for hidden nodes
    # b is the bais for visible nodes
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    
    # sample_h function is used for calculating probability of each hidden nodes given the value of visible nodes
    # x is visible nodes, wx is weights of visible nodes obtained by multiplication of visble nodes and transpose of weights.
    # Use sigmoid function to get the probability of each hidden node  
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    # sample_v function is used for calculating probability of each visible node given the value of hidden nodes. Rest is similiar to the above function
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    # v0 = initial visible node, vk = visible node after k samples, ph0 = initial hidden nodes, phk = hidden node after k samples
    # follow the standard training algorithm of Boltzmann machines
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
        
    def predict(self, x): # x: visible nodes
        _, h = self.sample_h(x)
        _, v = self.sample_v(h)
        return v

In [39]:
# Creating object of RBM class

nv = len(training_set[0])
nh = 500
batch_size = 500
rbm = RBM(nv, nh)

In [40]:
# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.3159)
epoch: 2 loss: tensor(0.2364)
epoch: 3 loss: tensor(0.2343)
epoch: 4 loss: tensor(0.2282)
epoch: 5 loss: tensor(0.2288)
epoch: 6 loss: tensor(0.2300)
epoch: 7 loss: tensor(0.2292)
epoch: 8 loss: tensor(0.2283)
epoch: 9 loss: tensor(0.2297)
epoch: 10 loss: tensor(0.2297)


In [41]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    # Original visible nodes required for sample_h
    v = training_set[id_user:id_user+1]
    # Testing visible nodes
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2210)


In [42]:
# Call predict function to get results of 'User-1' from the test set. Similarly we can get predicted movies results for other users too.
user_id = 2
user_input = Variable(test_set[user_id -1]).unsqueeze(0)
op = rbm.predict(user_input)
op = op.data.numpy()
ip_op = np.vstack([user_input, op])

In [43]:
import pandas as pd
f = pd.DataFrame(ip_op.T, columns = ['Original Ratings', 'Predicted Ratings'])
print('Ignore predictions where original ratings are -1 because that resembles no original value present for that movie')
print("\nUser-1's likely predicted ratings for all 1682 movies:\n")
print("Displaying movies' ratings where origial ratings are not -1:")
f[f['Original Ratings']!= -1.0]


Ignore predictions where original ratings are -1 because that resembles no original value present for that movie

User-1's likely predicted ratings for all 1682 movies:

Displaying movies' ratings where origial ratings are not -1:


,Original Ratings,Predicted Ratings
94,0.0,0.0
355,1.0,1.0
379,1.0,1.0
433,0.0,0.0
588,1.0,1.0
735,1.0,1.0
1095,1.0,1.0
1102,1.0,1.0
1187,1.0,0.0
1212,0.0,1.0
